In [72]:
import requests

In [94]:
hw_docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
hw_docs_response = requests.get(hw_docs_url)
hw_documents_raw = hw_docs_response.json()

hw_documents = []

for course in hw_documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        hw_documents.append(doc)

In [96]:
hw_documents[7]

{'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
 'section': 'General course-related questions',
 'question': 'Course - Can I follow the course after it finishes?',
 'course': 'data-engineering-zoomcamp'}

In [97]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')

In [98]:
es_client.info()

ObjectApiResponse({'name': 'a534715a6e8e', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'ftxcs0B1SESdcy1Q7L0FZA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [174]:
hw_index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

hw_index_name = "hw-course-questions"
#es_client.indices.delete(index=hw_index_name)
es_client.indices.create(index=hw_index_name, body=hw_index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'hw-course-questions'})

In [175]:
q = "How do copy a file to a Docker container?"

In [176]:
from tqdm.auto import tqdm

for doc in tqdm(hw_documents):
    es_client.index(index=hw_index_name,document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [177]:
def hw_elastic_search(query):

    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": q,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=hw_index_name, body=search_query)

    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source']) 

    return result_docs

In [178]:
hw_search_results = hw_elastic_search(q)

In [179]:
print(hw_search_results[2])

{'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I copy files from a different folder into docker container’s working directory?', 'course': 'machine-learning-zoomcamp'}


In [180]:
hw_context = ""

for doc in hw_search_results:
    hw_context = hw_context + f"Q: {doc['question']}\nA: {doc['text']}\n\n"

In [181]:
print(hw_context)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [184]:
def hw_build_prompt(query,search_results):

    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT:
    {context}
    """.strip()

    hw_context = ""

    for doc in search_results:
        hw_context = hw_context + f"Q: {doc['question']}\nA: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query,context=hw_context).strip()

    return prompt

In [185]:
print(len(hw_build_prompt(q,hw_elastic_search(q))))

1462


In [186]:
print(hw_build_prompt(q,hw_elastic_search(q)))

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: How do copy a file to a Docker container?

    CONTEXT:
    Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

In [187]:
!pip install tiktoken

In [189]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [193]:
tokens = encoding.encode(hw_build_prompt(q,hw_elastic_search(q)))

In [194]:
len(tokens)

324

In [195]:
def hw_llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{'role':'user','content':prompt}]
    )

    return response.choices[0].messages.content

In [196]:
def rag(query):
    search_results = hw_elastic_search(query)
    prompt = build_prompt(query,search_results)
    answer = llm(prompt)

    return answer

In [197]:
rag(q)

'You can copy a file to a Docker container using the `docker cp` command. The basic syntax is as follows:\n\n```bash\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\n```'

In [198]:
price_input = 0.005 # for 1k requests
price_output = 0.02 # for 1k requests

In [201]:
cost_150_input_tokens = 150 * price_input
cost_250_output_tokens = 250 * price_output

total_price = cost_150_input_tokens + cost_250_output_tokens

In [202]:
total_price

5.75